## Data

In [1]:
# import statements

import os
import pandas as pd
import numpy as np
import time
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta

import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

### Functions

In [2]:
# accepts betting pros cookies

def accept_cookies():
    
    try:
        element = driver.find_element("id","onetrust-accept-btn-handler")
        driver.execute_script("arguments[0].click();", element);
    except:
        pass

In [3]:
# get all player props for given day 

def get_props():
    
    props = {}
    
    try:
        
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
    
        rows = soup.find_all("div", {"id":"primary-info-container"})
        
        for row in rows:        
            s = str(row)
            soup = BeautifulSoup(s, "html.parser")
            name = soup.find_all("a", {"class":"pbcs__player-link"})[0].get_text().strip()
            line = float(soup.find_all("span", {"class":""})[2].get_text())
            props[name] = line
            
    except:
        
        pass
    
    return props

### Scraping BettingPros

In [4]:
# get all urls for the past year 

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

end_date = date.today()
start_date = end_date - relativedelta(months=1) # can change to years=1

urls = []
for single_date in daterange(start_date, end_date):
    url = 'https://www.bettingpros.com/nba/picks/prop-bets/bet/points?date={}'.format(single_date.strftime("%Y-%m-%d"))
    urls.append(url)

In [5]:
betting_df = pd.DataFrame(columns=['Name', 'Line', 'Date'])

for url in urls[-2:]:   
    
    r = requests.get(url)
    if r.status_code != 200: print("Error Loading", url)
        
    path_to_chromedriver = os.getcwd() + "/chromedriver"
    driver = webdriver.Chrome(executable_path=path_to_chromedriver)
        
    driver.get(url)
    
    accept_cookies()
    
    props = get_props()
    
    df = pd.DataFrame(props.items(), columns=['Name', 'Line'])
    df['Date'] = datetime.strptime(url[-10:], "%Y-%m-%d").date()
    betting_df = betting_df.append(df, ignore_index=True)
    
    driver.close()

<ipython-input-5-759ffe2cae1f>:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path_to_chromedriver)


### DataFrame to Google Sheets

In [7]:
betting_df.head(5)

,Name,Line,Date
0,Alperen Sengun,9.5,2023-01-28
1,Jayson Tatum,30.5,2023-01-28
2,Coby White,7.5,2023-01-28
3,Malcolm Brogdon,15.5,2023-01-28
4,Norman Powell,16.5,2023-01-28
